# OpenSearch hybrid search

Steps adapted from:
* https://opensearch.org/docs/latest/search-plugins/neural-search-tutorial/
* https://opensearch.org/docs/latest/search-plugins/hybrid-search/

## Configure ml plugin

In [7]:
curl -X PUT http://localhost:9200/_cluster/settings -H "Content-Type: application/json" -d'
{
  "persistent": {
    "plugins": {
      "ml_commons": {
        "only_run_on_ml_node": "false",
        "model_access_control_enabled": "true",
        "native_memory_threshold": "99"
      }
    }
  }
}'

{"acknowledged":true,"persistent":{"plugins":{"ml_commons":{"only_run_on_ml_node":"false","model_access_control_enabled":"true","native_memory_threshold":"99"}}},"transient":{}}


## Register a model group

In [2]:
curl -X POST http://localhost:9200/_plugins/_ml/model_groups/_register -H "Content-Type: application/json" -d'
{
  "name": "NLP_model_group",
  "description": "A model group for NLP models"
}'

{"model_group_id":"FvdfipIBXW2Cb22W17WN","status":"CREATED"}


## Register the model to the model group

In [8]:
curl -s -X POST http://localhost:9200/_plugins/_ml/models/_register -H "Content-Type: application/json" -d'
{
  "name": "huggingface/sentence-transformers/msmarco-distilbert-base-tas-b",
  "version": "1.0.1",
  "model_group_id": "FvdfipIBXW2Cb22W17WN",
  "model_format": "TORCH_SCRIPT"
}' | jq


{
  "task_id": "GPdgipIBXW2Cb22WobUC",
  "status": "CREATED"
}


## Check status of registering model

In [14]:
curl -s -X GET http://localhost:9200/_plugins/_ml/tasks/GPdgipIBXW2Cb22WobUC | jq

{
  "model_id": "GfdgipIBXW2Cb22Wp7XJ",
  "task_type": "REGISTER_MODEL",
  "function_name": "TEXT_EMBEDDING",
  "state": "COMPLETED",
  "worker_node": [
    "tVe5eJbgRv6-d-VACS1SfQ"
  ],
  "create_time": 1728898441461,
  "last_update_time": 1728898528997,
  "is_async": true
}


## Deploy the model

In [15]:
curl -s -X POST http://localhost:9200/_plugins/_ml/models/GfdgipIBXW2Cb22Wp7XJ/_deploy | jq

{
  "task_id": "GvdnipIBXW2Cb22WF7Uq",
  "task_type": "DEPLOY_MODEL",
  "status": "CREATED"
}


## Check the status

In [19]:
curl -s -X GET http://localhost:9200/_plugins/_ml/tasks/GvdnipIBXW2Cb22WF7Uq | jq
MODEL_ID=`curl -s -X GET http://localhost:9200/_plugins/_ml/tasks/GvdnipIBXW2Cb22WF7Uq | jq .model_id`
echo "Model ID is: ${MODEL_ID}"
export MODEL_ID=${MODEL_ID}

{
  "model_id": "GfdgipIBXW2Cb22Wp7XJ",
  "task_type": "DEPLOY_MODEL",
  "function_name": "TEXT_EMBEDDING",
  "state": "COMPLETED",
  "worker_node": [
    "tVe5eJbgRv6-d-VACS1SfQ"
  ],
  "create_time": 1728898864931,
  "last_update_time": 1728898888250,
  "is_async": true
}
Model ID is: "GfdgipIBXW2Cb22Wp7XJ"


## Create an ingest pipeline

In [20]:
curl -X PUT http://localhost:9200/_ingest/pipeline/nlp-ingest-pipeline -H "Content-Type: application/json" -d'
{
  "description": "A text embedding pipeline",
  "processors": [
    {
      "text_embedding": {
        "model_id": "GfdgipIBXW2Cb22Wp7XJ",
        "field_map": {
          "passage_text": "passage_embedding"
        }
      }
    }
  ]
}'

{"acknowledged":true}


## Create an index for ingestion

In [21]:
curl -X DELETE http://localhost:9200/my-nlp-index
curl -X PUT http://localhost:9200/my-nlp-index -H "Content-Type: application/json" -d'
{
  "settings": {
    "index.knn": true,
    "default_pipeline": "nlp-ingest-pipeline"
  },
  "mappings": {
    "properties": {
      "id": {
        "type": "text"
      },
      "passage_embedding": {
        "type": "knn_vector",
        "dimension": 768,
        "method": {
          "engine": "lucene",
          "space_type": "l2",
          "name": "hnsw",
          "parameters": {}
        }
      },
      "passage_text": {
        "type": "text"
      }
    }
  }
}'

{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [my-nlp-index]","index":"my-nlp-index","resource.id":"my-nlp-index","resource.type":"index_or_alias","index_uuid":"_na_"}],"type":"index_not_found_exception","reason":"no such index [my-nlp-index]","index":"my-nlp-index","resource.id":"my-nlp-index","resource.type":"index_or_alias","index_uuid":"_na_"},"status":404}
{"acknowledged":true,"shards_acknowledged":true,"index":"my-nlp-index"}


## Ingest documents into the index

In [22]:
curl -s -X PUT http://localhost:9200/my-nlp-index/_doc/1 -H "Content-Type: application/json" -d'
{
  "passage_text": "Hello world",
  "id": "s1"
}' | jq

curl -s -X PUT http://localhost:9200/my-nlp-index/_doc/2 -H "Content-Type: application/json" -d'
{
  "passage_text": "Hi planet",
  "id": "s2"
}' | jq

{
  "_index": "my-nlp-index",
  "_id": "1",
  "_version": 1,
  "result": "created",
  "_shards": {
    "total": 2,
    "successful": 1,
    "failed": 0
  },
  "_seq_no": 0,
  "_primary_term": 1
}
{
  "_index": "my-nlp-index",
  "_id": "2",
  "_version": 1,
  "result": "created",
  "_shards": {
    "total": 2,
    "successful": 1,
    "failed": 0
  },
  "_seq_no": 1,
  "_primary_term": 1
}


## Configure a search pipeline

In [23]:
curl -X PUT http://localhost:9200/_search/pipeline/nlp-search-pipeline -H "Content-Type: application/json" -d'
{
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              0.3,
              0.7
            ]
          }
        }
      }
    }
  ]
}'

{"acknowledged":true}


## Search the index using hybrid search

In [24]:
curl -s -X GET http://localhost:9200/my-nlp-index/_search?search_pipeline=nlp-search-pipeline -H "Content-Type: application/json" -d'
{
  "_source": {
    "exclude": [
      "passage_embedding"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "passage_text": {
              "query": "Hi world"
            }
          }
        },
        {
          "neural": {
            "passage_embedding": {
              "query_text": "Hi world",
              "model_id": "53y1SJEB62af-UG5cSSz",
              "k": 5
            }
          }
        }
      ]
    }
  }
}' | jq

{
  "error": {
    "root_cause": [
      {
        "type": "status_exception",
        "reason": "Failed to find model"
      }
    ],
    "type": "status_exception",
    "reason": "Failed to find model"
  },
  "status": 404
}


## Combining neural and match query

In [25]:
curl -s -X GET http://localhost:9200/my-nlp-index/_search?search_pipeline=nlp-search-pipeline -H "Content-Type: application/json" -d'
{
  "_source": {
    "exclude": [
      "passage_embedding"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "passage_text": {
              "query": "Hi world"
            }
          }
        },
        {
          "neural": {
            "passage_embedding": {
              "query_text": "Hi world",
              "model_id": "GfdgipIBXW2Cb22Wp7XJ",
              "k": 5
            }
          }
        }
      ]
    }
  }
}' | jq


{
  "took": 995,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 2,
      "relation": "eq"
    },
    "max_score": 1.0,
    "hits": [
      {
        "_index": "my-nlp-index",
        "_id": "2",
        "_score": 1.0,
        "_source": {
          "passage_text": "Hi planet",
          "id": "s2"
        }
      },
      {
        "_index": "my-nlp-index",
        "_id": "1",
        "_score": 0.3007,
        "_source": {
          "passage_text": "Hello world",
          "id": "s1"
        }
      }
    ]
  }
}
